In [ ]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
train = pd.read_csv('drive/My Drive/Colab Notebooks/Sentiment analysis/train.csv')
test = pd.read_csv('drive/My Drive/Colab Notebooks/Sentiment analysis/test.csv')

In [ ]:
train.head(10)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
5,6,1,iPhone software update fucked up my phone big ...
6,7,0,Happy for us .. #instapic #instadaily #us #son...
7,8,0,New Type C charger cable #UK http://www.ebay.c...
8,9,0,Bout to go shopping again listening to music #...
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...


In [ ]:
id = test['id']

In [ ]:
# train data
ps = PorterStemmer()
corpus = []
for i in range(7920):
  review =  re.sub('[^a-zA-Z]',' ',train['tweet'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
# test data
corpus_test = []
for i in range(len(test)):
  review =  re.sub('[^a-zA-Z]',' ',test['tweet'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus_test.append(review)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()
validation = cv.transform(corpus_test).toarray()

In [ ]:
y = train['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train,y_train)
y_preds = nb.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test,y_preds)

0.8520971302428256

In [ ]:
result = nb.predict(validation)

In [ ]:
submission = pd.read_csv('drive/My Drive/Colab Notebooks/Sentiment analysis/sample_submission.csv')
submission.head()

,id,label
0,7921,0
1,7922,0
2,7923,0
3,7924,0
4,7925,0


In [ ]:
submission['id'] = id
submission['label'] = result

In [ ]:
submission.to_csv('sub1.csv',index=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_preds = lr.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test,y_preds)

0.8169014084507042

# XgBoost

In [ ]:
params = {
    'learning_rate' : [0.05, 0.10,0.15,0.20,0.25,0.30],
    'max_depth': [3,4,6,8,10],
    'min_child_weight': [1,3,5,7],
    'gamma': [0.0,0.1,0.2,0.5],
    'colsample_bytes':[0.3,0.4,0.5,0.7]
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier

In [ ]:
clf = XGBClassifier(learning_rate=0.10,max_depth=5)
clf.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_preds = clf.predict(X_test)
f1_score(y_test,y_preds)

0.8348214285714286

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
f1_score(y_test,y_pred)

0.8205128205128205